<a href="https://colab.research.google.com/github/alexisnpavlik/Binance-Futures-Calculator/blob/main/_SONIC_Alexis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [1]:
# Prepare the environment
%%capture
from google.colab import drive, auth
drive.mount('/content/drive')
%cd /content/drive/MyDrive/python-library
!pip install .
!sudo apt-get install python-pymysql
!pip install colab-env
!pip install Hashids
import colab_env
%cd /content
import nltk
nltk.download('stopwords')
import gallifrey
import pandas as pd
import numpy as np
import time
from hashids import Hashids

#Set up for Hash calculation
hasher = Hashids("jS5qAlR0kHbAeqeuVowPBnyotKuTvvlGzIzkEtz79XQGB7BJHb", 
                 8, 
                 "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")

# Authenticate
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())

# create, and save df
from gspread_dataframe import set_with_dataframe

## Query


In [2]:
main_query= """

SELECT
    -- ID
    o.id as ID,
    -- Job title
    o.objective as 'Job title',
    -- location
    (select group_concat(l.location) from opportunity_places l where l.opportunity_id = o.id and l.active = 1) as 'location',
    -- Created date 
    DATE(o.created) as 'Created date',
    -- Approved date
    DATE(o.reviewed) as 'Approved date',
    -- Commit date
    DATE(och.created) as 'Commited date',
    -- Account Manager
    p.username as 'Account manager',
    -- Type of job
    o.commitment_id as 'Type of job',
    -- Type of service
    o.fulfillment as 'Type of service',
    -- Language of the post
    o.locale as 'Language of the post',
    -- Status
    o.status as 'Status',
    -- Mutual matches
    sum(case when me.interested is not null and oc.interested is not null then 1 else 0 end) as 'Mutual Matches',
    -- Disqualified
    sum(case when me.send_disqualified_notification is not null then 1 else 0 end)  as 'Disqualified',
    -- Changes history, last updated
    DATE(o.last_updated) as 'Last changes',
    -- Closing date
    DATE(o.deadline) as 'Closing Date',
    -- Completed applications
    sum(case when oc.interested is not null then 1 else 0 end) as 'Completed Applications',
    -- Incomplete applications
    sum(case when oc.created is not null then 1 else 0 end) as 'Incomplete applications',
    -- Others
    sum(case when oc.id is not null and oc.interested is not null and oc.column_id is not null
    and oc2.name <> 'mutual matches'
    and (last_evaluation.last_interest is not null and (last_evaluation.last_not_interest is null or last_evaluation.last_interest > last_evaluation.last_not_interest)) then 1 else 0 end)
    as 'Others',
    -- Active
    sum(case when me.interested is not null and oc.interested is not null then 1 else 0 end) + sum(case when oc.id is not null and oc.interested is not null and oc.column_id is not null
    and oc2.name <> 'mutual matches'
    and (last_evaluation.last_interest is not null and (last_evaluation.last_not_interest is null or last_evaluation.last_interest > last_evaluation.last_not_interest)) then 1 else 0 end)  as 'Active',
    -- Pending for review
    ( sum(case when oc.interested is not null then 1 else 0 end) - ( sum(case when me.interested is not null and oc.interested is not null then 1 else 0 end) + sum(case when oc.id is not null and oc.interested is not null and oc.column_id is not null
    and oc2.name <> 'mutual matches'
    and (last_evaluation.last_interest is not null and (last_evaluation.last_not_interest is null or last_evaluation.last_interest > last_evaluation.last_not_interest)) then 1 else 0 end) + me.send_disqualified_notification ) ) as 'pending for review',
    -- Hires
    DATE(osh.hiring_date) as 'Hires',
    -- Company
   (select organization_id from opportunity_organizations where opportunity_id =  o.id  group by organization_id limit 1) as 'Company_id',
    -- Sharing token
    (select sharing_token from opportunity_members where manager = true and status = 'accepted' and opportunity_id =  o.id  limit 1) as 'Sharing token'

FROM opportunities as o

    -- Necessary for completed applications and incomplete applications
    join opportunity_candidates as oc on o.id = oc.opportunity_id
    -- Necessary for account manager
    join opportunity_changes_history as och on o.id = och.opportunity_id
    -- Necessary for account_mannager
    left join person_account_managers as pam on och.person_id = pam.person_id
    -- join to make the relation people-person_account_mannager
    left join people as p on pam.person_id = p.id
    -- join to asign osh to get match dates
    left join opportunity_stats_hires osh on o.id = osh.opportunity_id
    -- join to get member evaluation
    join member_evaluations as me on oc.id = me.candidate_id
    -- Join to get the correct relation into id's 
    left join opportunity_columns oc2 on oc.column_id = oc2.id
    -- join to get calculations
    left join ( select me.candidate_id, max(me.interested) as last_interest, max(me.not_interested) as last_not_interest
    from member_evaluations me group by me.candidate_id) last_evaluation on last_evaluation.candidate_id = oc.id
      
    

WHERE true
    #and me.send_disqualified_notification = true
    and o.objective <> 'Shared by an intermediary'
    and o.review = 'approved'
    -- Approved date is not null
    and o.reviewed is not null
    and osh.hiring_date is not null
    and p.username in ('catalinazarate12','juanfebog','jpinilla','cappadaniela27','laumariareyest','laurammedinag','diego19_franco35','inglisscardenas')

group by o.id
order by o.created desc;

"""

In [3]:
organizations_query = """

SELECT 
  id as 'Company_id',
  name as 'Company' 
  
FROM organizations;

"""

## Manual Update

In [4]:
main_df = pd.DataFrame(gallifrey.discovery.execute_query(main_query))
organizations_df = pd.DataFrame(gallifrey.solar.execute_query(organizations_query))

In [5]:
main_df.head(10)

,ID,Job title,location,Created date,Approved date,Commited date,Account manager,Type of job,Type of service,Language of the post,Status,Mutual Matches,Disqualified,Last changes,Closing Date,Completed Applications,Incomplete applications,Others,Active,pending for review,Hires,Company_id,Sharing token
0,1782830,Desarrollador Flutter,None,2022-01-24,2022-01-25,2022-01-26,juanfebog,full-time,prime,en,open,22.0,75.0,2022-01-29,2022-02-23,49.0,75.0,16.0,38.0,10.0,2022-01-28,730852,FMZ2lb5M8xpY
1,1782796,Analista de Sistemas (Bogotá),"Bogotá, Colombia",2022-01-24,2022-01-24,2022-01-24,jpinilla,full-time,prime,en,closed,40.0,130.0,2022-01-29,2022-01-29,125.0,130.0,40.0,80.0,45.0,2022-01-25,332263,58oGUmUh88Xs
2,1714727,Sales Director,None,2022-01-14,2022-01-14,2022-01-14,jpinilla,part-time,prime,en,closed,15.0,159.0,2022-01-25,2022-01-25,132.0,159.0,6.0,21.0,110.0,2022-01-24,381186,ni3qJ6oFFc0O
3,1704695,Recruiting Manager,None,2022-01-13,2022-01-13,2022-01-14,jpinilla,full-time,prime,es,closed,77.0,1463.0,2022-01-27,2022-01-25,854.0,1463.0,7.0,84.0,769.0,2022-01-27,785782,cH9UXWSrHpi2
4,1704675,Ingeniero de Desarrollo .NetCore,Colombia,2022-01-13,2022-01-14,2022-01-15,jpinilla,full-time,prime,es,open,45.0,246.0,2022-01-29,2022-02-12,81.0,246.0,21.0,66.0,14.0,2022-01-27,740780,1xQ19v3tdqDs
5,1689657,Client Success Coordinator,None,2022-01-11,2022-01-12,2022-01-13,juanfebog,full-time,prime,en,closed,42.0,102.0,2022-01-27,2022-01-19,81.0,102.0,37.0,79.0,2.0,2022-01-26,960840,O0TmBhG3lKtd
6,1689655,Desarrollador PHP,None,2022-01-11,2022-01-11,2022-01-11,juanfebog,full-time,prime,es,closed,11.0,74.0,2022-01-28,2022-01-28,34.0,74.0,5.0,16.0,17.0,2022-01-28,443430,cmz2NSxRt5PS
7,1689649,Tech leader,Colombia,2022-01-11,2022-01-11,2022-01-11,juanfebog,full-time,prime,en,closed,9.0,60.0,2022-01-24,2022-01-24,19.0,60.0,6.0,15.0,4.0,2022-01-18,730852,YqnMK9ZRfSJa
8,1679604,Profesor de alemán,None,2022-01-10,2022-01-10,2022-01-11,jpinilla,full-time,prime,es,open,12.0,276.0,2022-01-29,2022-02-09,54.0,276.0,18.0,30.0,23.0,2022-01-29,797801,hURJOLUckzT8
9,1662078,Auxiliar de Cocina,"Bogotá, Colombia",2022-01-07,2022-01-17,2022-01-17,jpinilla,full-time,prime,es,open,18.0,73.0,2022-01-29,2022-02-06,73.0,73.0,11.0,29.0,44.0,2022-01-11,332263,5tyIb4sL7406


In [5]:
organizations_df.head(10)

,Company_id,Company
0,795908,
1,773085,0 2 Convergencia Digital S A S
2,763230,2 Way Sports
3,795148,23Ky Colombia S.A.S.
4,891553,60 Decibels
5,760653,Aberdare College
6,761403,"Absolute Computer Solutions, Inc."
7,805948,ABUSA Lab
8,786038,Academia de Inglés de la Universidad de Tarapacá
9,786905,Academia de Música DEUS


In [6]:
# URL Generator
def url_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)

# Manager URL Generator
def manager_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)+'/members-list?sharingToken='

# Transforms columns
main_df.insert(9, "URL", main_df['ID'].apply(url_generator),True)
main_df.insert(10, "Manager link", main_df['ID'].apply(manager_generator),True)
main_df['Manager link']=main_df['Manager link'].apply(lambda x:x+main_df['Sharing token'])

In [7]:
opportunities_df = pd.merge(left=main_df,right=organizations_df).drop(['Company_id'],axis=1)

opportunities_df = opportunities_df.reindex(columns=['ID','Job title','Company','location','Created date','Approved date','Commited date','Account manager','Type of job',
                                             'Type of service','URL','Manager link','Language of the post','Status','Mutual Matches','Disqualified','Last changes','Closing Date',
                                             'Completed Applications','Incomplete applications','Others','Active','pending for review','Hires'])

# Update Google Sheet

In [12]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

set_with_dataframe(sheet, opportunities_df) 

In [10]:
sheet.clear()


NameError: ignored

# Save to Google drive

##### Save csv

In [ ]:
drive.mount('/content/drive')

In [ ]:
opportunities_df.to_csv('/content/drive/My Drive/Data_studio_dataset/Dataset.csv',index=False,encoding='utf-8')

##### Save  Excel

In [ ]:
drive.mount('/content/drive')

In [ ]:
opportunities_df.to_excel('/drive/My Drive/Data_studio_dataset/Dataset.xls',index=False,encoding='utf-8')

# Auto update (update required)


In [14]:
# Time in minutes
minutes_for_update = 30

In [ ]:
# URL Add
def link_add(id):
  id_alfa=hasher.encode(id)
  return 'https://torre.co/postings/'+id_alfa

In [ ]:
while True:
  Opportunities = pd.DataFrame(gallifrey.discovery.execute_query(query))

  Opportunities.insert(9, "URL", Opportunities['ID'].apply(link_add), True) 
  
  sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1
  set_with_dataframe(sheet, Opportunities)

  for i in range(minutes_for_update):
    time.sleep(60)
    print('Tiempo restante hasta la proxima actualización:',minutes_for_update-i,'minutos')

  print('Updating...')

Tiempo restante hasta la proxima actualización: 30 minutos


KeyboardInterrupt: ignored

# Testing

In [82]:
query_testing = """

select id as 'Company_id', name as 'Company' from organizations ;

"""

In [83]:
Testing = pd.DataFrame(gallifrey.solar.execute_query(query_testing))

### Update Google Sheet

In [88]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

set_with_dataframe(sheet, query_testing) 

In [84]:
Testing.head()

,Company_id,Company
0,795908,
1,773085,0 2 Convergencia Digital S A S
2,763230,2 Way Sports
3,795148,23Ky Colombia S.A.S.
4,891553,60 Decibels
